In [1]:
import numpy as np
import cv2
import tensorflow as tf
import keras
from tensorflow.python.distribute.mirrored_strategy import MirroredStrategy

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
#from keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (ConvLSTM2D, BatchNormalization, Convolution3D, Convolution2D, Conv2D, Dropout,
                          TimeDistributed, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Input,  AveragePooling3D)

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import gc

2023-03-30 18:57:23.617131: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ['CUDA_VISIBLE_DEVICES'] = "2"
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow.keras.backend as K

In [4]:
data = np.load("/home/sullah/PycharmProjects/EDSR/Alphorn_Project/dataset_after_augmentation_256x256/del_data_after_augmentation_256x256.npy", mmap_mode='r+')

In [5]:
# load the saved model
model_encoder = tf.keras.models.load_model('ConvLSTM_with_Pooling_with_MSE_autoencoder_decoder-encoder_fully_ConvLSTM.h5', compile = False)
model_decoder = tf.keras.models.load_model('ConvLSTM_with_Pooling_with_MSE_autoencoder_decoder-decoder_fully_ConvLSTM.h5', compile = False)

2023-03-30 18:57:25.959240: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-03-30 18:57:25.960487: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-03-30 18:57:26.046028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2023-03-30 18:57:26.048019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2023-03-30 18:57:26.048911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:5e:00.0 name: GeFo

In [6]:
model_encoder.summary()
model_decoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256, 256, 1)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 32, 256, 256, 32)  38144     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 32, 128, 128, 32)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 128, 128, 32)  128       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 32, 128, 128, 64)  221440    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 32, 64, 64, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 64, 64, 64)    256 

In [7]:
predictions = model_encoder.predict(data, batch_size=1)

2023-03-30 18:57:50.068849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-30 18:57:50.088201: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2023-03-30 18:57:50.621711: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:592] layout failed: Invalid argument: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/convolution_7' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/add_6', 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/Tanh_1' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_5', 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_2' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/add_5', 'encoder/conv_lst_m2d_1/while/body/_49/encoder/conv_lst_

2023-03-30 19:01:15.296770: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8200 of size 512 next 2157
2023-03-30 19:01:15.296796: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8400 of size 512 next 1358
2023-03-30 19:01:15.296815: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8600 of size 768 next 2212
2023-03-30 19:01:15.296832: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8900 of size 256 next 1662
2023-03-30 19:01:15.296850: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8a00 of size 256 next 2075
2023-03-30 19:01:15.296867: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8b00 of size 256 next 434
2023-03-30 19:01:15.296884: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8c00 of size 256 next 1427
2023-03-30 19:01:15.296914: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f10c34e8d00

ResourceExhaustedError:  OOM when allocating tensor with shape[1,32,32,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node encoder/max_pooling3d/MaxPool3D (defined at tmp/ipykernel_2148386/2170329185.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_predict_function_3259]

Function call stack:
predict_function


In [ ]:
print(predictions.shape)

In [ ]:
np.save('encoder_predictions', predictions)

# As this code/predictions of all frames with give an error with the GPU memory, so I have predicted few frames in each step in files 3.1 and concatenated them all in file 3.2. 